In [2]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/3. Analyses/2. Data Processing/Output'
cur_date = '030425'

library(missForest)
library(readxl)
library(openxlsx)
library(tidyverse)
library(imputeLCMD)
library(factoextra)

# reading in files
ws_df = data.frame(Data = 'WS', read_excel("Input/Woodsmoke_Data_012825.xlsx", sheet = 2)) %>%
    select(-Sample_Number)
wf_df = data.frame(Data = 'WF', read_excel("Input/Wildfire_Data_012825.xlsx", sheet = 2)) %>%
    select(-Sample_Number)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
an

NOTES TO SELF:
- change 'replicate' to 'instance' and 'chemical class' to 'class'

In [3]:
head(ws_df)
head(wf_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,1263480,Burnet et al. 1990,NA,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WF,1289821,Liang et al. 2021,1,PAH,Volume,NA,1-(10-methylanthracen-9-yl)ethanone,5
2,WF,1289821,Liang et al. 2021,1,PAH,Volume,DTXSID50176885,1-Acenaphthenone,1
3,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
4,WF,1289739,Campos et al. 2019,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
5,WF,1289739,Campos et al. 2019,NA,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
6,WF,1289739,Campos et al. 2019,NA,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA


In [4]:
dim(ws_df)
dim(wf_df)

[1] 6607    9

[1] 4340    9

# Filter 1

Removing duplicate records for instances that weren't measured at all (MAR). 

In [5]:
`%notin%` <- Negate(`%in%`)

ws_df = ws_df %>%
    filter(Replicate %notin% NA)

wf_df = wf_df %>%
    filter(Replicate %notin% NA)

In [6]:
dim(ws_df)
dim(wf_df)

[1] 4203    9

[1] 2378    9

In [7]:
# combining dfs
combined_df = rbind(ws_df, wf_df)
head(combined_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,267091,Forchhammer et al. 2012,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


In [8]:
length(unique(combined_df$Name))

[1] 192

192 unique variables.

# Chemical/Metal Background Filters

Determining what variables have missing data for all variables delineated based on the dfs the data will be split into and analyzed. 

A variable (ie. `Name` which represents a chemical or metal) will be retained if >= 50% of its experimental data points were measurable values. In other words, chemicals were excluded if they were NAs (not evaluated) or had non-detect values across > 50% of the samples.

In [9]:
sample_type_presence_df = combined_df %>%
    # if an experimental value is there count it as being "present"
    mutate(count = ifelse(Value %notin% c('NA', 'ND'), 1, 0)) %>%
    # determining which have at least one experimental value within each sample type
    group_by(HAWC_ID, Metric, Name) %>%
    # calculating the percentage of experimental data points
    reframe(Variable_Presence_Percentage = (sum(count)/n()) * 100) %>%
    filter(Variable_Presence_Percentage >= 50)

head(sample_type_presence_df)

HAWC_ID,Metric,Name,Variable_Presence_Percentage
<dbl>,<chr>,<chr>,<dbl>
266980,Volume,Acetaldehyde,100
266980,Volume,Benzene,100
266980,Volume,Benzo(GHI)perylene,100
266980,Volume,Benzo(a)pyrene,100
266980,Volume,Benzo(b)fluoranthene,100
266980,Volume,Benzo(k)fluoranthene,100


In [48]:
# filtering the original df
filter1_df = inner_join(sample_type_presence_df[,1:3], combined_df) %>%
    # metals don't have a DTXSID, so making that col their name
    mutate(DTXSID = ifelse(DTXSID != 'NA', DTXSID, Name)) 

head(filter1_df)

Joining with `by = join_by(HAWC_ID, Metric, Name)`


HAWC_ID,Metric,Name,Data,Study,Replicate,Chemical_Class,DTXSID,Value
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
266980,Volume,Acetaldehyde,WS,Stockfelt et al. 2012,1,VOC,DTXSID5039224,71000
266980,Volume,Acetaldehyde,WS,Stockfelt et al. 2012,2,VOC,DTXSID5039224,37000
266980,Volume,Benzene,WS,Stockfelt et al. 2012,1,VOC,DTXSID3039242,33000
266980,Volume,Benzene,WS,Stockfelt et al. 2012,2,VOC,DTXSID3039242,21000
266980,Volume,Benzo(GHI)perylene,WS,Stockfelt et al. 2012,1,PAH,DTXSID5023908,41
266980,Volume,Benzo(GHI)perylene,WS,Stockfelt et al. 2012,2,PAH,DTXSID5023908,11


In [49]:
dim(combined_df)
dim(filter1_df)

[1] 6581    9

[1] 4189    9

Started with 6581 records, 2392 were removed, leaving 4189. 

# Second Chemical/Metal Background Filter

In the intial filter, WS and WF samples were combined. However, this time variables (ie. metal or chemical) will be split based on their `Metric` (ie. volume or weight) and then retained if that variable is found in both woodsmoke and wildfire samples within volume or weight samples.

In [50]:
split_filtered_df = filter1_df %>%
    group_by(Data, Metric) %>%
    group_split

split_ws_vol_df = split_filtered_df[[3]]
split_ws_weight_df = split_filtered_df[[4]]
split_wf_vol_df = split_filtered_df[[1]]
split_wf_weight_df = split_filtered_df[[2]]

In [51]:
# seeing how many unique variables are in each df and if they're consistent in each file
length(unique(split_ws_vol_df$Name))
length(unique(split_wf_vol_df$Name))
length(unique(split_ws_weight_df$Name))
length(unique(split_wf_weight_df$Name))

[1] 98

[1] 68

[1] 94

[1] 70

In [52]:
# they're not so first getting variables that are in weight or volume samples
consistent_wf_vol_df = split_wf_vol_df %>%
    filter(Name %in% unique(split_ws_vol_df$Name))
consistent_ws_vol_df = split_ws_vol_df %>%
    filter(Name %in% consistent_wf_vol_df$Name)
consistent_wf_weight_df = split_wf_weight_df %>%
    filter(Name %in% unique(split_ws_weight_df$Name))
consistent_ws_weight_df = split_ws_weight_df %>%
    filter(Name %in% consistent_wf_weight_df$Name)

length(unique(consistent_wf_vol_df$Name))
length(unique(consistent_ws_vol_df$Name))
length(unique(consistent_wf_weight_df$Name))
length(unique(consistent_ws_weight_df$Name))

[1] 33

[1] 33

[1] 34

[1] 34

There were 98, 68, 94 and 70 woodsmoke weight, woodsmoke volume, wildfire weight, and wildfire volume samples, respectively. 33 variables were common between volume samples and 34 were common between weight records and will be retained.

In [53]:
# recombining data
vol_df = rbind(consistent_ws_vol_df, consistent_wf_vol_df)
weight_df = rbind(consistent_ws_weight_df, consistent_wf_weight_df)

head(vol_df)

HAWC_ID,Metric,Name,Data,Study,Replicate,Chemical_Class,DTXSID,Value
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
266980,Volume,Benzene,WS,Stockfelt et al. 2012,1,VOC,DTXSID3039242,33000
266980,Volume,Benzene,WS,Stockfelt et al. 2012,2,VOC,DTXSID3039242,21000
266980,Volume,Benzo(GHI)perylene,WS,Stockfelt et al. 2012,1,PAH,DTXSID5023908,41
266980,Volume,Benzo(GHI)perylene,WS,Stockfelt et al. 2012,2,PAH,DTXSID5023908,11
266980,Volume,Benzo(k)fluoranthene,WS,Stockfelt et al. 2012,1,PAH,DTXSID0023909,23
266980,Volume,Benzo(k)fluoranthene,WS,Stockfelt et al. 2012,2,PAH,DTXSID0023909,4.5


# QRILC Imputation

Imputing non-detect data using QRILC didn't work, so I'm just replacing the non-detect data with zeros.

In [54]:
# determining number of ND and NA values in each df
vol_df %>%
    filter(Value %in% c('NA', 'ND')) %>%
    group_by(Value) %>%
    summarize(n())

weight_df %>%
    filter(Value %in% c('NA', 'ND')) %>%
    group_by(Value) %>%
    summarize(n())

Value,n()
<chr>,<int>


Value,n()
<chr>,<int>
NA,2
ND,34


In [55]:
dim(vol_df)
dim(weight_df)

[1] 744   9

[1] 1411    9

Only the weight dataframe had non-detect values representing MNAR data. The remaining MAR data from the weight df will be imputed using random forest (RF).

In [56]:
mar_weight_df = weight_df %>%
    filter(Value == 'NA')

preimputed_df = anti_join(weight_df, mar_weight_df) 
preimputed_df$Value = as.numeric(preimputed_df$Value)
mar_weight_df$Value = as.numeric(mar_weight_df$Value)

# reordering
preimputed_df = preimputed_df[,c(1,2,4:8,3,9)]

head(preimputed_df)

Joining with `by = join_by(HAWC_ID, Metric, Name, Data, Study, Replicate,
Chemical_Class, DTXSID, Value)`
Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”


HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,PAH,DTXSID4020878,2-Methylphenanthrene,10.0
267105,Weight,WS,Danielsen et al. 2011,2,PAH,DTXSID4020878,2-Methylphenanthrene,37.0
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cr,Cr,21.4
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cr,Cr,21.3
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cu,Cu,65.6
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cu,Cu,34.4


Gave up on QRILC imputation for all mnar data, since I didn't have enough instances to impute within each study. I also couldn't impute across the entire dataset given that the number of data instances for each study wasn't the same. Therefore, sample groups were assigned a value of 0 if they had two or fewer instances.

In [57]:
# creating a df for mnar data
mnar_df = preimputed_df %>%
    filter(is.na(Value)) 

mnar_df[c(9:12,15,22:27,30:34),9] = 0
head(mnar_df)

HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
271638,Weight,WS,Jalava et al. 2012,4,PAH,DTXSID3021774,Acenaphthene,NA
271638,Weight,WS,Jalava et al. 2012,6,PAH,DTXSID3021774,Acenaphthene,NA
271638,Weight,WS,Jalava et al. 2012,7,PAH,DTXSID3021774,Acenaphthene,NA
271638,Weight,WS,Jalava et al. 2012,6,PAH,DTXSID5047740,Coronene,NA
271769,Weight,WS,Uski et al. 2012,4,PAH,DTXSID3021774,Acenaphthene,NA
271769,Weight,WS,Uski et al. 2012,6,PAH,DTXSID3021774,Acenaphthene,NA
271769,Weight,WS,Uski et al. 2012,7,PAH,DTXSID3021774,Acenaphthene,NA
271769,Weight,WS,Uski et al. 2012,6,PAH,DTXSID5047740,Coronene,NA
298728,Weight,WS,Tapanainen et al. 2011,1,PAH,DTXSID3021774,Acenaphthene,0


In [58]:
QRILC_imputation = function(dataset){
    set.seed(12)
      # """
    # Creating a quantile normalization function to normalize each sample.
    # :param (input): exposed and unexposed (vehicle) dfs
    # :output: 1 quantile normalized df
    # """
    #ADD WORDS
    wider_dataset = dataset %>% 
        # removing these cols temporarily 
        select(-c("Chemical_Class", "Name")) %>%
        pivot_wider(names_from = DTXSID, values_from = Value)

    # normalizing data since that what the QRILC function wants
    # had to pseudo log transform to prevent Inf values
    QRILC_prep = wider_dataset[,6:ncol(wider_dataset)] %>%
         mutate_all(., function(x) log10(x + 1)) %>%
         as.matrix()
                        
    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.1)
    QRILC_log10_df = data.frame(imputed_QRILC_object[1]) 
    
    # converting back the original scale
    QRILC_df = QRILC_log10_df %>%
        mutate_all(., function(x) 10^x - 1)
    
    imputed_dataset = data.frame(cbind(unique(dataset[,1:5]), QRILC_df)) %>%
         pivot_longer(cols = 6:ncol(wider_dataset), names_to = "DTXSID", values_to = "Value")         

    return(imputed_dataset)

}

In [59]:
# will impute within each study
df_for_ids = mnar_df[is.na(mnar_df$Value),]
study_ids = unique(df_for_ids$HAWC_ID)

imputed_df = data.frame()
for (i in 1:length(study_ids)){

    filtered_df = preimputed_df %>%
        filter(HAWC_ID == study_ids[i])

    # calling fn
    imp_df = QRILC_imputation(filtered_df)
    imputed_df = rbind(imputed_df, imp_df)
    
}
# adding back in additional cols
imputed_df = inner_join(unique(imputed_df), unique(weight_df[,1:8]))
imputed_df = imputed_df[,c(1:5,9,6,8,7)]

head(imputed_df)

Joining with `by = join_by(HAWC_ID, Metric, Data, Study, Replicate, DTXSID)`


HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID3021774,Acenaphthene,132.7
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID4075459,Benzo(c)phenanthrene,493.9
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID8052691,Benzo(j)fluoranthene,617.3
271638,Weight,WS,Jalava et al. 2012,1,Metal,Ca,Ca,1140.0
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID5047740,Coronene,75.5
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID6062591,Cyclopenta(cd)pyrene,1067.0


In [65]:
imputed_weight_df = rbind(imputed_df, 
                          # adding back in non-imputed data
                          preimputed_df %>%
                              filter(HAWC_ID %notin% unique(mnar_df$HAWC_ID)),
                          # adding back in mar data
                          mar_weight_df, 
                          # adding back in mnar data that was converted to zeros
                          mnar_df[!is.na(mnar_df$Value),])

head(imputed_weight_df)
dim(weight_df)
dim(imputed_weight_df)

HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID3021774,Acenaphthene,132.7
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID4075459,Benzo(c)phenanthrene,493.9
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID8052691,Benzo(j)fluoranthene,617.3
271638,Weight,WS,Jalava et al. 2012,1,Metal,Ca,Ca,1140.0
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID5047740,Coronene,75.5
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID6062591,Cyclopenta(cd)pyrene,1067.0


[1] 1411    9

[1] 1248    9

In [ ]:
# length(unique(imputed_df$HAWC_ID))
# length(unique(mar_weight_df$HAWC_ID))
# bleh = mnar_df[!is.na(mnar_df$Value),]
# length(unique(bleh$HAWC_ID))    
# bleh2 = drop_na(preimputed_df) %>%
#                                filter(HAWC_ID %notin% unique(mnar_df$HAWC_ID))
# length(unique(bleh2$HAWC_ID))    
# length(unique(weight_df$HAWC_ID))

# x = c(unique(imputed_df$HAWC_ID), unique(mar_weight_df$HAWC_ID),unique(bleh$HAWC_ID), unique(bleh2$HAWC_ID))
# x[duplicated(x)]

# weight_df %>%
#     filter(HAWC_ID %in% x[duplicated(x)][1])

In [89]:
# dim(imputed_df)
# dim(mar_weight_df)
# dim(mnar_df[!is.na(mnar_df$Value),])
# dim(drop_na(preimputed_df) %>%
#                               filter(HAWC_ID %notin% unique(mnar_df$HAWC_ID)))

[1] 335   9

[1] 2 9

[1] 14  9

[1] 943   9

In [20]:
# avg_df = preimputed_df %>%
#     group_by(HAWC_ID, Name) %>%
#     mutate(Avg_Value = mean(Value)) %>%
#     select(-c("Value", "Replicate")) %>%
#     unique()

# head(avg_df)

# RF Imputation

In [67]:
random_forest_imputation = function(dataset){
    # ADD WORDS
    wider_dataset = dataset %>%
        # removing these cols temporarily 
        select(-c("Chemical_Class", "Name")) %>%
        pivot_wider(names_from = DTXSID, values_from = Value)
    
    preimputed_df = wider_dataset[,6:ncol(wider_dataset)]
    # had to drop rows (studies) that only had missing data
    #df = preimputed_df[rowSums(is.na(preimputed_df)) != ncol(preimputed_df), ]
    
    imputed_RF_object = missForest(as.matrix(df))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(wider_dataset[,1:5], imputed_RF_df)) %>%
        pivot_longer(cols = 6:ncol(wider_dataset), names_to = "DTXSID", values_to = "Value")
    
    return(imputed_final_df)
}


# calling fn
random_forest_imputation(imputed_weight_df %>% 
                             filter(HAWC_ID == unique(mar_weight_df$HAWC_ID)))

ERROR: Error in as.vector(x, mode): cannot coerce type 'closure' to vector of type 'any'


In [20]:
# # exporting
# write.xlsx(processed_df, paste0(Output,"/", "Processed_Data_", cur_date, ".xlsx"), 
#            rowNames = FALSE)